In [1]:
import rasterio

In [2]:
import numpy as np

In [3]:
from scipy.sparse import lil_matrix

In [4]:
from scipy.spatial.distance import euclidean

In [5]:
 def graph_from_dem(dem_path, connectivity=4):
     
    """
    Builds a graph from a LiDAR-derived DEM (TIFF)
    
    Args: 
        dem_path (str): Path to the DEM TIFF file
        connectivity (int): assign neighbors (4 for cardinal, 8 for card. + diags)
    
    Returns:
        gruple: A tuple containing:
            - graph (scipy.sparse.lil_matrix): Adjacency matrix of the graph
            - node_coords (dict): Dictionary mapping node index to (row, col) coords
            - dem_data (numpy.ndarray): the DEM data as a 2D numpy array
    """
    
    with rasterio.open(dem_path) as src:
        dem_data = src.read(1)
        height, width = dem_data.shape
    
    num_nodes = height*width
    graph = lil_matrix((num_nodes, num_nodes))
    node_coordinates = {}
    
    # define neighbors from connectivity
    if connectivity == 4:
        neighbors = [(-1,0), (1,0), (0,1), (0,-1)] # cardinal
    elif connectivity == 8:
        neighbors = [(-1,0), (1,0), (0,1), (0,-1),
                     (-1,-1), (-1,1), (1,-1), (1,1)] # card + diag
    else:
        raise ValueError("connectivity must be 4 or 8")
    
    for r in range(height):
        for c in range(width):
            current_node_idx = r * width + c # remember it is zero-indexed!
            node_coordinates[current_node_idx] = (r,c)
    
            for dr, dc in neighbors:
                nr, nc = r + dr, c + dc
    
                if 0 <= nr < height and 0 <= nc < width:
                    neighbor_node_idx = nr*width + nc
    
                    # calculate edge weight (e.g., elevation difference, Euclidean dist)
                    # here, use elevation diff.
                    weight = abs(dem_data[r,c] - dem_data[nr,nc])
    
                    # alt. spatial distance
                    # pixel_size_x, pixel_size_y = src.res
                    # dist = euclidean([c*pixel_size_x, r*pixel_size_y],
                    #                 [nc*pixel_size_x, nr*pixel_size_y])
                    # weight = dist * (abs(dem_data[r,c] - dem_data[nr, nc]) +1) # example
                    graph[current_node_idx, neighbor_node_idx] = weight
    
    return graph, node_coordinates, dem_data

In [6]:
# r convert normal string to raw string
dem_file = "19TDJ409849.tif"

In [7]:
graph, node_coords, dem_data = graph_from_dem(dem_file, connectivity=4)